In [1]:
import os
import cv2
import numpy as np
import pickle
import random
import tensorflow as tf
from keras import Sequential
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Flatten
from sklearn.model_selection import train_test_split
from keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.333
session = InteractiveSession(config=config)

2025-02-11 17:03:31.180268: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-11 17:03:31.197597: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739273611.218434 2693032 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739273611.224641 2693032 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-11 17:03:31.246774: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [ ]:
tf.config.run_functions_eagerly(True)

In [ ]:
categories = ['with_mask', 'without_mask']

In [ ]:
data = []
for category in categories:
    path = os.path.join('train',category)
    for file in os.listdir(path):
        img_path = os.path.join(path,file)
        label = categories.index(category)
        img = cv2.imread(img_path)
        img = cv2.resize(img,(224,224))
        data.append([img,label])

In [ ]:
random.shuffle(data)

In [ ]:
X = []
y = []
for features,label in data:
    X.append(features)
    y.append(label)

In [ ]:
X = np.array(X)/ 255.0
y = np.array(y)

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2, random_state=42)

In [ ]:
# X_train.shape

In [ ]:
# X_test.shape

In [ ]:
vgg = VGG16(include_top=False, input_shape=(224, 224, 3))

In [ ]:
# vgg.summary()

In [ ]:
# Build model
model = Sequential()
model.add(vgg)
model.add(Flatten())  # Convert feature maps to 1D
model.add(Dense(256, activation='relu'))  # Optional extra layer
model.add(Dense(1, activation='sigmoid'))

In [ ]:
# model.summary()

In [ ]:
for layer in vgg.layers:
    layer.trainable = False
model.summary()

In [ ]:
model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [ ]:
model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test), callbacks=[early_stopping])

In [ ]:
# pickle.dump(model,open('model.pkl','wb'))

In [2]:
model = pickle.load(open("model.pkl",'rb'))

I0000 00:00:1739273616.166720 2693032 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 16173 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:51:00.0, compute capability: 8.6


In [3]:
def predict_face_mask(img, model):
    img = img.astype("float32") / 255.0  # Normalize
    img = np.expand_dims(img, axis=0)  # Add batch dimension

    y_pred = model.predict(img)
    confidence = y_pred[0][0]  # Get prediction confidence

    print(f"[DEBUG] Prediction: {y_pred} (Confidence: {confidence:.2f})")

    if confidence > 0.5:
        return "Without Mask", confidence
    else:
        return "With Mask", 1 - confidence  # Adjust confidence for "With Mask"

In [4]:
# write in video
def draw_label(img,text,pos,bg_color):
    text_size = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 1, cv2.FILLED)

    end_x = pos[0] + text_size[0][0] + 2
    end_y = pos[1] + text_size[0][1] - 2

    cv2.rectangle(img,pos,(end_x,end_y),bg_color,cv2.FILLED)
    cv2.putText(img, text, pos, cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,0),1,cv2.LINE_AA)

In [5]:
# for rectangle around face
import cv2
haar = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
def detect_face(img):
    coordindates = haar.detectMultiScale(img)
    return coordindates

In [6]:
# Function to detect mask in video
def detect_mask_in_video(video_path, model):
    cap = cv2.VideoCapture(video_path)

    # Get video properties
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))


    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = haar.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=7) 

        if len(faces) == 0:
            print("[INFO] No faces detected in this frame.")

        for (x, y, w, h) in faces:
            # Ignore small detections (likely false positives)
            if w < 50 or h < 50:
                print(f"[INFO] Ignoring small detection: x={x}, y={y}, w={w}, h={h}")
                continue  

            face = frame[y:y+h, x:x+w]
            face = cv2.resize(face, (224, 224))

            prediction = predict_face_mask(face, model)

            print(f"[DEBUG] Face detected at: x={x}, y={y}, w={w}, h={h} - Prediction: {prediction}")

            color = (0, 255, 0) if prediction == "With Mask" else (0, 0, 255)
            cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
            cv2.putText(frame, prediction, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

        cv2.imshow("Mask Detection", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    if output_path:
        out.release()
    cv2.destroyAllWindows()

# Run detection on a video file
detect_mask_in_video('test2.mp4', model)


[INFO] No faces detected in this frame.
[INFO] No faces detected in this frame.
[INFO] No faces detected in this frame.
[INFO] No faces detected in this frame.
[INFO] No faces detected in this frame.
[INFO] No faces detected in this frame.
[INFO] No faces detected in this frame.
[INFO] No faces detected in this frame.
[INFO] No faces detected in this frame.
[INFO] No faces detected in this frame.
[INFO] No faces detected in this frame.
[INFO] No faces detected in this frame.
[INFO] No faces detected in this frame.
[INFO] No faces detected in this frame.
[INFO] No faces detected in this frame.
[INFO] No faces detected in this frame.
[INFO] No faces detected in this frame.
[INFO] No faces detected in this frame.
[INFO] No faces detected in this frame.
[INFO] No faces detected in this frame.
[INFO] No faces detected in this frame.
[INFO] No faces detected in this frame.
[INFO] No faces detected in this frame.
[INFO] No faces detected in this frame.
[INFO] No faces detected in this frame.


I0000 00:00:1739273641.271209 2949434 service.cc:148] XLA service 0x73269400a290 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1739273641.271292 2949434 service.cc:156]   StreamExecutor device (0): NVIDIA RTX A6000, Compute Capability 8.6
2025-02-11 17:04:01.298085: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1739273641.384166 2949434 cuda_dnn.cc:529] Loaded cuDNN version 90300


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


I0000 00:00:1739273642.389431 2949434 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


[DEBUG] Prediction: [[0.7268591]] (Confidence: 0.73)
[DEBUG] Face detected at: x=305, y=35, w=70, h=70 - Prediction: ('Without Mask', 0.7268591)


error: OpenCV(4.10.0) :-1: error: (-5:Bad argument) in function 'putText'
> Overload resolution failed:
>  - Can't convert object to 'str' for 'text'
>  - Can't convert object to 'str' for 'text'


In [ ]:
def detect_mask_with_webcam(model):
    cap = cv2.VideoCapture(0)  # Open webcam
    haar = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

    while True:
        ret, frame = cap.read()
        if not ret:
            print("[ERROR] Failed to capture image from webcam.")
            break

        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = haar.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5)

        if len(faces) == 0:
            print("[INFO] No faces detected.")

        for (x, y, w, h) in faces:
            face = frame[y:y+h, x:x+w]
            face = cv2.resize(face, (224, 224))

            prediction, confidence = predict_face_mask(face, model)

            # Debugging: Print face coordinates and confidence score
            print(f"[DEBUG] Face at x={x}, y={y}, w={w}, h={h} - {prediction} ({confidence:.2f})")

            color = (0, 255, 0) if prediction == "With Mask" else (0, 0, 255)
            label = f"{prediction} ({confidence:.2f})"  # Add confidence score

            cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
            cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

        cv2.imshow("Mask Detection - Webcam", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to exit
            break

    cap.release()
    cv2.destroyAllWindows()

# Run mask detection from webcam
detect_mask_with_webcam(model)


In [ ]:
cap = cv2.VideoCapture(0)

In [ ]:
while True:
    ret, frame = cap.read()
    print(ret, frame)

    img = cv2.resize(frame,(224,224))
    y_pred = predict_face_mask(img)
    coords = detect_face(cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY))

    for x,y,h,w in coords:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),3)
    if(y_pred=="With Mask"):
        draw_label(frame,"With Mask",(10,10),(0,255,0))
    else:
        draw_label(frame,"Without Mask",(10,10),(0,0,255))
    cv2.imshow("window",frame)

    if cv2.waitKey(1) & 0xFF == ord('x'):
        break

cv2.destroyAllWindows()

In [ ]:
sample1 = cv2.imread('mask.jpg')
sample1 = cv2.resize(sample1,(224,224))

In [ ]:
predict_face_mask(sample1,model)

In [ ]:
sample2 = cv2.imread('without_mask.jpg')
sample2 = cv2.resize(sample2,(224,224))

In [ ]:
predict_face_mask(sample2, model)

In [ ]:
sample3 = cv2.imread('test11.jpg')
sample3 = cv2.resize(sample3,(224,224))
sample4 = cv2.imread('test12.jpg')
sample4 = cv2.resize(sample4,(224,224))
sample5 = cv2.imread('test13.jpg')
sample5 = cv2.resize(sample5,(224,224))

In [ ]:
sample6 = cv2.imread('test21.jpg')
sample6 = cv2.resize(sample6,(224,224))
sample7 = cv2.imread('test22.jpg')
sample7 = cv2.resize(sample6,(224,224))
sample8 = cv2.imread('test23.jpg')
sample8 = cv2.resize(sample6,(224,224))

In [ ]:
predict_face_mask(sample3)

In [ ]:
predict_face_mask(sample4)

In [ ]:
predict_face_mask(sample5)

In [ ]:
predict_face_mask(sample6)

In [ ]:
predict_face_mask(sample7)

In [ ]:
predict_face_mask(sample8)

In [ ]:
import cv2
import numpy as np

def predict_face_mask(img, model):
    img = img.astype("float32") / 255.0  # Normalize
    img = np.expand_dims(img, axis=0)  # Add batch dimension

    y_pred = model.predict(img)
    confidence = y_pred[0][0]  # Get prediction confidence

    print(f"[DEBUG] Prediction: {y_pred} (Confidence: {confidence:.2f})")

    if confidence > 0.5:
        return "Without Mask", confidence
    else:
        return "With Mask", 1 - confidence  # Adjust confidence for "With Mask"

def detect_mask_with_webcam(model):
    cap = cv2.VideoCapture(0)  # Open webcam
    haar = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

    while True:
        ret, frame = cap.read()
        if not ret:
            print("[ERROR] Failed to capture image from webcam.")
            break

        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = haar.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5)

        if len(faces) == 0:
            print("[INFO] No faces detected.")

        for (x, y, w, h) in faces:
            face = frame[y:y+h, x:x+w]
            face = cv2.resize(face, (224, 224))

            prediction, confidence = predict_face_mask(face, model)

            # Debugging: Print face coordinates and confidence score
            print(f"[DEBUG] Face at x={x}, y={y}, w={w}, h={h} - {prediction} ({confidence:.2f})")

            color = (0, 255, 0) if prediction == "With Mask" else (0, 0, 255)
            label = f"{prediction} ({confidence:.2f})"  # Add confidence score

            cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
            cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

        cv2.imshow("Mask Detection - Webcam", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to exit
            break

    cap.release()
    cv2.destroyAllWindows()

# Run mask detection from webcam
detect_mask_with_webcam(model)
